In [ ]:
#Functional API
from keras import Input,layers

input_tensor = Input(shape=(32,)) #建立一個輸入張量
print(input_tensor.shape)

dense = layers.Dense(16,activation='relu') #建立一個Dense層，並將其想像成一個函數
output_tensor = dense(input_tensor) # 將張量輸入層函數，他會回傳經處理後的結果張亮
print(output_tensor.shape)

In [ ]:
#序列式(Sequential)建立模型
from keras.models import Sequential,Model
from keras import layers,Input

model=Sequential()
model.add(layers.Dense(32,activation='relu',input_shape=(64,)))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(32,activation='softmax'))

model.summary()

In [ ]:
#函數式(API)建立模型
#透過建立Model物件
input_tensor = Input(shape=(64,)) # 建立一個初始張量

x = layers.Dense(32,activation='relu')(input_tensor)

y = layers.Dense(32,activation='relu')(x)

output_tensor = layers.Dense(10,activation='softmax')(y)

model = Model(input_tensor,output_tensor)
model.summary()

In [ ]:
#如果用完全不相干的輸入和輸出張量去建構模型。
#因為Keras找不到相關資訊，導致執行時會生錯誤。
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input,output_tensor)

In [ ]:
#在編譯、訓練或驗證此Model物件時，API的功能與序列式模型相同
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
import numpy as np

x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

# 將訓練輸入模型進行訓練
model.fit(x_train,y_train,epochs=10,batch_size=128)
score = model.evaluate(x_train,y_train)
print(score)

In [ ]:
#多輸入模型
from keras import Model 
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,),dtype='int32',name='text')
embedded_text = layers.Embedding(text_vocabulary_size,64)(text_input)
print(embedded_text.shape)
encoded_text = layers.LSTM(32)(embedded_text)
print(encoded_text.shape)

question_input = Input(shape=(None,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size,32)(question_input)
print(embedded_question.shape)
encoded_question = layers.LSTM(16)(embedded_question)
print(encoded_question.shape)

concatenated = layers.concatenate([encoded_question,encoded_text],axis=1)
print(concatenated.shape)

answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)
print(answer.shape)

model = Model([text_input,question_input],answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

In [ ]:
#訓練雙輸入模型的方法
import numpy as np

num_samples = 1000
max_length = 100

# 產生虛擬text資料：1000筆，每筆100個字(數字)
text = np.random.randint(1,text_vocabulary_size,size = (num_samples,max_length))
print(text.shape)

# 產生虛擬question資料：1000筆，每筆100個字(數字)
question = np.random.randint(1,question_vocabulary_size,size = (num_samples,max_length))
print(question.shape)

# 產生虛擬answer資料：1000筆，每筆100個字(數字)
answers = np.zeros(shape=(num_samples,answer_vocabulary_size),dtype='int32')

for answer in answers:
  answer[np.random.randint(answer_vocabulary_size)]=1
print(answers.shape)

# 訓練方法1：使用list將送入資料進行訓練
model.fit([text,question],answers,epochs=10,batch_size=128)

# 訓練方法2；使用dict將送入資料進行訓練，鍵為Input層的名稱，值為Numpy的值
model.fit({'text':text,'question':question},answers,epochs=10,batch_size=128)

In [ ]:
#多輸出模型
from keras import layers,Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10 #將收入分成10群

posts_input = Input(shape=(None,),dtype='int32',name='posts')

# 用函數式API將輸入向量傳入Embedding層，得到維度為256的崁入向量
embedding_posts = layers.Embedding(vocabulary_size,256)(posts_input)
print(embedding_posts.shape)

# 以函數式API將砍入向量傳入一層層之中處理
x = layers.Conv1D(128,5,activation='relu')(embedding_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128,activation='relu')(x)
print(x.shape) # 走過一連串層後，x.shape為(?,128)

#======================================================================#
#將x向量分別送到3個輸入層
#======================================================================#
# 1. 預測年紀的輸出層：純量迴歸任務
age_prediction = layers.Dense(1,name='age')(x)

# 2. 預測收入族群的輸出層：多分類任務
income_prediction = layers.Dense(num_income_groups,activation='softmax',name='income')(x)

# 3. 預測性別的輸出層：二元分類任務
gender_prediction = layers.Dense(1,activation='softmax',name='gender')(x)

#======================================================================#
#用輸入向量與輸出向量實例化Model物件
#======================================================================#
model = Model(posts_input,[age_prediction,income_prediction,gender_prediction])

model.summary()

In [ ]:
#1x1卷積 (瓶頸層)
from keras import layers,Input

x = Input(batch_shape=(1000,28,28,256))
print(x.shape)

branch_a = layers.Conv2D(64,1,activation='relu',strides=2)(x) #使用步長參數=2的進行1/2採樣
print("分支a",branch_a.shape)

branch_b = layers.Conv2D(64,1,activation='relu')(x) # 進行1x1逐點卷積，故shape大小不變
branch_b = layers.Conv2D(128,1,activation='relu',strides=2,padding='same')(branch_b) # 進行3X3空間卷積，並使用步長參數=2 進行1/2採樣
print("分支b",branch_b.shape)

branch_c = layers.AveragePooling2D(3,strides=2,padding='same')(x) # 採樣發生在平均池化中
branch_c = layers.Conv2D(128,1,activation='relu',padding='same')(branch_c) 
print("分支c",branch_c.shape)

branch_d = layers.Conv2D(128,1,activation='relu')(x)
branch_d = layers.Conv2D(128,3,activation='relu',padding='same')(branch_d) # 進行3X3空間卷積，並使用步長參數=2 進行1/2採樣
branch_d = layers.Conv2D(128,3,activation='relu',strides=2,padding='same')(branch_d) # 進行3X3空間卷積，並使用步長參數=2 進行1/2採樣
print("分支d",branch_d.shape)

#================================================================#
#串接分支輸出以取得模組輸出
output = layers.concatenate([branch_a,branch_b,branch_c,branch_d],axis=-1)
print(output.shape)

In [ ]:
#線性轉換 Linear Transformations
#存在一個函數(也可以視為矩陣) T 可以將 R^m 空間的向量(張量)對應(轉換)到 R^n 空間的向量(張量),只要 R^n 空間的任意向量 u v符合以下兩個條件即可稱此轉換為線性轉換：

#可加性
#齊次性
from keras import layers,Input

x = Input(batch_shape=(1000,32,32,128)) # 定義4D張量 x
y = layers.Conv2D(128,3,activation='relu',padding='same')(x)
z = layers.Conv2D(128,3,activation='relu',padding='same')(y)

op = layers.add([z,x])
print(op.shape)

In [ ]:
from keras import layers,Input

x = Input(batch_shape=(1000,32,32,128)) # 定義4D張量 x
y = layers.Conv2D(128,3,activation='relu',padding='same')(x)
z = layers.Conv2D(128,3,activation='relu',padding='same')(y)
print(z.shape)

t = layers.MaxPool2D(2,strides=2)(z)
print(t.shape)

residual = layers.Conv2D(128,1,strides=2,padding='same')(x) # 對張量進行線性轉換以縮小採樣，並將channel降低盛與張量t相同的128
print(residual.shape)

op = layers.add([t,residual])
print(op.shape)

In [ ]:
#孿生LSTM (Siamese LSTM)
from keras import layers,Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None,128))
print(left_input.shape)
left_output = lstm(left_input)
print(left_output.shape)

right_input = Input(shape=(None,128))
print(right_input.shape)
right_output = lstm(right_input)
print(right_output.shape)

merged = layers.concatenate([left_output,right_output],axis=-1) # 將向量串接
print(merged.shape)

predictions = layers.Dense(1,activation='sigmoid')(merged)
model = Model([left_input,right_input],predictions)

In [ ]:
#不需要兩個獨立模型自左右鏡頭萃取視覺特徵，可以藉由同一個層來一起進行萃取
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None,include_top=False)

left_input = Input(shape=(250,250,3))
right_input = Input(shape=(250,250,3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

print(left_features.shape)
print(right_features.shape)

merged_features = layers.concatenate([left_features,right_features],axis=-1)

In [ ]:
#EarlyStopping 與 ModelCheckpoint 回呼
import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_acc',patience=1),
    keras.callbacks.ModelCheckpoint(filepath='my_model.ht',monitor='val_acc',save_best_only=True)
]

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

model.fit(x,y,validation_data=(x_val,y_val),epochs=10,batch_sizes=32,callbacks=callbacks_list)

In [ ]:
#RedeuceLROnPlateau 回呼
import keras

callbacks_list = [keras.callbacks.RedeuceLROnPlateau(monitor='val_loss',factor=0.1,patience=10)]

model.fit(x,y,validation_data=(x_val,y_val),epochs=10,batch_sizes=32,callbacks=callbacks_list)

In [ ]:
#撰寫自己的回呼
import keras 
import numpy as np 

class ActivationLogger(keras.callbacks.Callback): # 繼承Callback類別

    def set_model(self,model):
        self.model = model
        layer_outputs = [layers.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input,layer_outputs)

    def on_epoch_end(self,epoch,logs=None):
        if self.validation is None:
            raise RuntimeError(Requires validation_data')

        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('validation_at_epoch',str(epoch)+'.npz','w')
        np.save(f,activations)
        f.close()

In [ ]:
#使用Tensor內建的文字分類模型
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train,maxlen=max_len)
x_test = sequence.pad_sequences(x_test,maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features,128,input_length=max_len,name='embed'))
model.add(layers.Conv1D(32,7,activation='relu'))
model.add(layers.MaxPool1D(5))
model.add(layers.Conv1D(32,7,activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1))
model.summary()

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
#使用TensorBoard回呼來訓練模型
callbacks_list = [
    keras.callbacks.TensorBoard(log_dir='my_log_dir',histogram_freq=1,embeddings_freq=1)
]

history = model.fit(x_train,y_train,epochs=20,batch_size=128,validation_split=0.2,callbacks=callbacks_list)

In [ ]:
from keras.utils import plot_model
plot_model(model,to_file='model.png')

In [ ]:
from keras.utils import plot_model
plot_model(model,show_shapes=True,to_file='model.png')